In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_8/models/B5R2b5_AN_1FC_fold8_3.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold8_3.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(7368, 22)
(7368, 22)
Normal:  (3991, 22)
Abnormal:  (3377, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267
1,1,11,11,15,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,257.0,103.0,538.0,634.0,0.256617,0.115278,0.537205,0.705051
2,2,17,17,15,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,SevereFattyLiver,https://irisprodseatraining.blob.core.windows....,0.0,184.0,573.0,751.0,0.000862,0.190025,0.638344,0.773737
3,3,21,21,15,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,278.0,87.0,515.0,496.0,0.278395,0.097096,0.515427,0.551515
4,4,23,23,15,P1,P1,Abnormal,AB05,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF1,https://irisprodseatraining.blob.core.windows....,249.0,69.0,439.0,385.0,0.254909,0.076894,0.448980,0.428283


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 7368 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.0735315 , -0.13731213,  0.28273287, ..., -0.07504269,
         0.13403927, -0.12755033],
       [-0.17032348, -0.14423771,  0.23478547, ...,  0.29679239,
         0.30950379, -0.09680816],
       [-0.18144928, -0.15285897, -0.15769456, ...,  0.14844061,
         0.06868927, -0.15815806],
       ...,
       [ 0.60856318,  0.30984938, -0.06154582, ..., -0.1276563 ,
         0.07614338,  0.37564963],
       [ 0.02199187,  0.1299879 , -0.18701075, ..., -0.16540097,
        -0.10733981,  0.15148546],
       [ 0.63472885,  0.56351143, -0.06431263, ..., -0.07432333,
        -0.07889527,  0.55074692]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.073532,-0.137312,0.282733,0.083720,-0.203224,-0.184130,0.162200,-0.194846,-0.158898,-0.074829,...,-0.148398,0.110808,-0.179616,-0.140071,-0.207968,-0.038988,-0.015148,-0.075043,0.134039,-0.127550
1,-0.170323,-0.144238,0.234785,0.296260,-0.217903,-0.189379,0.163410,-0.234621,-0.220291,-0.019988,...,-0.121092,0.182835,-0.192608,-0.172707,-0.183245,-0.161039,0.128365,0.296792,0.309504,-0.096808
2,-0.181449,-0.152859,-0.157695,0.033461,-0.200052,-0.151878,0.117279,-0.150576,-0.211502,-0.069214,...,-0.174614,-0.041255,-0.173543,-0.210108,-0.140477,-0.171491,-0.061372,0.148441,0.068689,-0.158158
3,-0.182013,-0.175779,0.170395,0.457328,-0.209899,-0.095732,1.000316,-0.232229,-0.212870,0.281321,...,-0.189803,0.780215,-0.217112,-0.173906,-0.219182,0.773911,0.453935,0.538463,0.781271,-0.169567
4,-0.154250,-0.202830,-0.081837,-0.045834,-0.220811,-0.191688,0.034673,-0.193362,-0.203003,-0.121654,...,-0.060530,0.099531,-0.190472,-0.165070,-0.222572,-0.011915,-0.128062,-0.084202,-0.144768,-0.156361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7363,0.843284,0.441933,-0.145207,0.115132,0.695095,1.011106,-0.145128,0.783063,0.800580,-0.215071,...,0.541885,-0.176234,0.779190,0.330576,0.997947,-0.148869,0.133946,-0.123196,0.051774,0.582146
7364,0.628452,0.497133,0.008442,0.204830,0.782213,0.986520,-0.130831,0.840254,1.018736,-0.220214,...,0.448193,-0.215396,0.627091,0.413776,0.487129,-0.186995,0.061770,-0.160484,0.098390,0.153022
7365,0.608563,0.309849,-0.061546,-0.053633,0.718535,1.010926,-0.078436,0.934963,0.677534,-0.210907,...,0.274009,-0.210830,0.704411,0.494531,0.788859,-0.187995,0.173746,-0.127656,0.076143,0.375650
7366,0.021992,0.129988,-0.187011,-0.144503,0.134347,0.178286,-0.117127,0.238295,0.205396,-0.191095,...,0.011016,-0.193294,0.518514,0.211479,0.086458,-0.142173,-0.167849,-0.165401,-0.107340,0.151485


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.073532,-0.137312,0.282733,0.083720,-0.203224,-0.184130,0.162200,-0.194846,-0.158898,-0.074829,...,-0.207968,-0.038988,-0.015148,-0.075043,0.134039,-0.127550,Abnormal,P1,FP-A,AB01
1,-0.170323,-0.144238,0.234785,0.296260,-0.217903,-0.189379,0.163410,-0.234621,-0.220291,-0.019988,...,-0.183245,-0.161039,0.128365,0.296792,0.309504,-0.096808,Abnormal,P1,FP-A,AB02
2,-0.181449,-0.152859,-0.157695,0.033461,-0.200052,-0.151878,0.117279,-0.150576,-0.211502,-0.069214,...,-0.140477,-0.171491,-0.061372,0.148441,0.068689,-0.158158,Abnormal,P1,FP-A,AB03
3,-0.182013,-0.175779,0.170395,0.457328,-0.209899,-0.095732,1.000316,-0.232229,-0.212870,0.281321,...,-0.219182,0.773911,0.453935,0.538463,0.781271,-0.169567,Abnormal,P1,FP-A,AB04
4,-0.154250,-0.202830,-0.081837,-0.045834,-0.220811,-0.191688,0.034673,-0.193362,-0.203003,-0.121654,...,-0.222572,-0.011915,-0.128062,-0.084202,-0.144768,-0.156361,Abnormal,P1,FP-A,AB05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7363,0.843284,0.441933,-0.145207,0.115132,0.695095,1.011106,-0.145128,0.783063,0.800580,-0.215071,...,0.997947,-0.148869,0.133946,-0.123196,0.051774,0.582146,Normal,P8,FP-E,Normal
7364,0.628452,0.497133,0.008442,0.204830,0.782213,0.986520,-0.130831,0.840254,1.018736,-0.220214,...,0.487129,-0.186995,0.061770,-0.160484,0.098390,0.153022,Normal,P8,FP-E,Normal
7365,0.608563,0.309849,-0.061546,-0.053633,0.718535,1.010926,-0.078436,0.934963,0.677534,-0.210907,...,0.788859,-0.187995,0.173746,-0.127656,0.076143,0.375650,Normal,P8,FP-E,Normal
7366,0.021992,0.129988,-0.187011,-0.144503,0.134347,0.178286,-0.117127,0.238295,0.205396,-0.191095,...,0.086458,-0.142173,-0.167849,-0.165401,-0.107340,0.151485,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold8_3.csv') #เปลี่ยนชื่อไฟล์